<span style="color:blue">Copyright (c) 2014-2019 National Technology and Engineering
Solutions of Sandia, LLC. Under the terms of Contract DE-NA0003525
with National Technology and Engineering Solutions of Sandia, LLC,
the U.S. Government retains certain rights in this software.</span>    
    
<span style="color:blue">Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions
are met:</span>    
    
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:green">1. Redistributions of source code must retain the above copyright
   notice, this list of conditions and the following disclaimer.</span>    
    
&nbsp;&nbsp;&nbsp;&nbsp;<span style="color:green">2. Redistributions in binary form must reproduce the above copyright
   notice, this list of conditions and the following disclaimer in the
   documentation and/or other materials provided with the distribution.</span>    
    
<span style="color:blue">THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY
THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

**Purpose:** Sample code to assemble points into trajectories    
In some cases, you may wish to read in trajectories with certain constraints. For example, we can have trajectories with a minimum number of points. Or we acknowledge that the points in the trajectory should be within a certain time and/or distance threshold to belong to the same trajectory. The Trajectory Builder does this.

In [3]:
from tracktable.source.trajectory import AssembleTrajectoryFromPoints
from tracktable.core import geomath
from tracktable.domain import all_domains as ALL_DOMAINS
import importlib
import itertools
import datetime

<span style="color:blue">Set up a trajectory builder using the specified parameters</span>    
    
Instantiate and configure an AssembleTrajectoryFromPoints. You will still need to set its 'input' member before it's ready to go.    
    
<span style="color:orange">Args:</span>    
&nbsp;&nbsp;&nbsp;&nbsp;separation_distance: Two successive points that are at least this far apart (in KM) will be placed in separate trajectories.    
&nbsp;&nbsp;&nbsp;&nbsp;separation_time: Two successive points with at least this many minutes between them will be placed in separate trajectories.    
&nbsp;&nbsp;&nbsp;&nbsp;minimum_length: Trajectories with fewer than this many points will be discarded.    
        
<span style="color:orange">Returns:</span>    
&nbsp;&nbsp;&nbsp;&nbsp;A new instance of AssembleTrajectoryFromPoints

In [4]:
def configure_trajectory_builder(separation_distance=100, separation_time=20, minimum_length=10):
    source = AssembleTrajectoryFromPoints()
    if separation_distance is not None:
        source.separation_distance = float(separation_distance)
    if separation_time is not None:
        source.separation_time = datetime.timedelta(minutes=separation_time)
    if minimum_length is not None:
        source.minimum_length = minimum_length
    return source

**Requirements:** In order to use the trajectory builder, we need a source that gives us data points in time sequential order. As with before, we can use the Trajectory Reader.

In [5]:
inFile = open('../data/SampleFlight.csv')
domain = 'terrestrial'                 # we want to create a terrestrial point reader
if domain.lower() not in ALL_DOMAINS:  #Format domain and make sure it is an available domain
    raise KeyError("Domain '{}' is not in list of installed domains ({}).".format(domain, ', '.join(ALL_DOMAINS)))
else:
    domain_to_import = 'tracktable.domain.{}'.format(domain.lower())
    domain_module = importlib.import_module(domain_to_import)

point_source = domain_module.TrajectoryPointReader()
point_source.input = inFile
point_source.comment_character = '#'
point_source.field_delimiter = ','

Now that we have a time sequential source of data points, we can apply constraints to build trajectories.

In [6]:
source = configure_trajectory_builder(separation_distance=50, separation_time=10, minimum_length=5)

Now we can apply the source of our points (the point reader we created above) and generate the trajectories by iterating over them. You will notice the output confirms the contrainsts we set in the previous step. Also, note only a single trajectory was created.

In [7]:
source.input = point_source
trajectories = source.trajectories()

for traj in trajectories:
    print(traj)   

INFO: Trajectory assembly: New trajectories will be declared after a separation of 50.0 units or 0:10:00 seconds.
INFO: Trajectories with fewer than 5 points will be rejected.
INFO: Done assembling trajectories.


Great! We got a single trajectory since all the points met our constraints.  

Based on the constraints we've already created, we can generate multiple trajectories from a single data set.
In the next sample set of data, two points are seperated by a significant amount of time.

KKK011,2004-12-07 17:31:18,-82.0381,26.9886,3500,97,135,0  
KKK011,2004-12-16 12:13:05,-81.9747,27.0306,3300,168,351,0

Ideally this should end one trajectory and start a new one. 

In [8]:
inFile = open('../data/TwoSampleFlights.csv')
point_source.input = inFile
source = configure_trajectory_builder(separation_distance=50, separation_time=10, minimum_length=5)

source.input = point_source
trajectories = source.trajectories()

for traj in trajectories:
    print(traj)   

INFO: Trajectory assembly: New trajectories will be declared after a separation of 50.0 units or 0:10:00 seconds.
INFO: Trajectories with fewer than 5 points will be rejected.
INFO: Done assembling trajectories.
